# setup

In [8]:
(ns r-interop
    (:require [rojure.core :refer [get-r r-set! r-eval r-get with-r-eval]]
              [rojure.convert]
              [gg4clj.core]
              [json-html.core :refer [edn->html]]
              [clojupyter.misc.display :refer [make-html]]
              [hiccup.core :as hiccup]
              [clojure.java.io :as io]))


## connection
define connection to R (needs running R with RServe started)


In [2]:
(def r-conn (get-r))


#'r-interop/r-conn

## auxiliary functions


running r code through Rojure with given bindings to clojure values, returning the result as a clojure value

In [3]:
(defn call-r-with-code [bindings code]
    (doseq [[k v] bindings]
        (r-set! r-conn (name k) v))
    (r-eval r-conn
            "result <- NULL")
    (r-eval r-conn
            (str "result <- {" code "}"))
    (r-get r-conn "result"))

#'r-interop/call-r-with-code

using gg4clj to write r code as a clojure data structure

In [4]:
(defn call-r-as-data [bindings code-as-data]
    (->> code-as-data
         gg4clj.core/to-r
         (call-r-with-code bindings)))

#'r-interop/call-r-as-data

# simple usage example

## linear regression

In [5]:
(-> {:x [1 2 3 4 5 6]
     :y [10.5 19 31 39 49.5 62]} 
    (call-r-with-code "lm(y~x)")
    (get "coefficients"))

[-0.5333333333333405 10.200000000000001]

## statistics of random numbers


In [6]:
(-> {:n [1000]
     :mu [3]
     :sigma [4]}
    (call-r-as-data [:as.list [:summary
                      [:+ :mu [:'*' :sigma [:rnorm :n]]]]])
    edn->html
    make-html)

1st Qu. 0 0.20504836193973075 3rd Qu. 0 5.744968668087701 Max. 0 16.12268256342119 Mean 0 3.0587579194217427 Median 0 3.0992815193236005 Min. 0 -10.938969266393046

## plotting

This requires to install the R-packages ggplot2, rmarkdown.
TODO: Make this work again.

In [9]:
;; (def rmd-path "/tmp/tmp1.Rmd")
;; (def html-path "/tmp/tmp1.html")
;; (io/delete-file rmd-path)
;; (io/delete-file html-path)

;; (->> [[:library :ggplot2]
;;       [:+ [:ggplot [:data.frame {:x :x :y :y}]]
;;           [:geom_point [:aes {:x :x :y :y}]]]]
;;     gg4clj.core/to-r
;;     (format "```{r}\n%s\n```")
;;     (spit rmd-path))

;; (-> {:x [4 1 5 12 4 1 5 1]
;;      :y [1 23 22 2 3 5 6 9]}
;;     (call-r-as-data [[:library :rmarkdown]
;;                      [:render rmd-path {:output_file html-path}]]))

;; (->> html-path
;;      slurp
;;      make-html)
